Section: Thông tin
1. MSSV: 20520934
2. Họ tên: Bùi Quốc Thịnh
3. Bài tập: 4
4. Ngày thực hiện: 21/03/2022

##Yêu cầu 1: Phân lớp ảnh dogs vs cats

In [97]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [98]:
import os

dir_dataset = '/gdrive/MyDrive/CS231.M21.KHCL/Baitap/Dataset/dogscats_small'
dir_datatrain = os.path.join(dir_dataset, 'train')
dir_datatest = os.path.join(dir_dataset, 'test')

In [99]:
import cv2 
import numpy as np
from scipy import ndimage
from google.colab.patches import cv2_imshow
import os
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

In [100]:
def CalHist(pathfilename):
  img = cv2.imread(pathfilename,0)
  hist = cv2.calcHist([img], [0], None, [256], [0,256])
  size = img.shape[0]*img.shape[1]
  hist = hist / size
  return hist

In [101]:
from skimage import feature

def compute_hog_train(pathfilename):
  img = cv2.imread(pathfilename,0)
  img = cv2.resize(img, (128, 256))
  (hog, hog_image) = feature.hog(img, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), block_norm='L2-Hys', visualize=True, transform_sqrt=True)

  return hog

In [102]:
def EF(pathfilename, op):
  if op == 0:
    return CalHist(pathfilename)
  else: 
    return compute_hog_train(pathfilename)

In [103]:
def ReadData(path_to_files, op):
  features = []
  labels = []
  for file_name in os.listdir(path_to_files):
    # Take label of the image
    label = file_name.split(os.path.sep)[-1].split(".")[0]
    # -> Insert into list of labels
    if label == 'dog':
      labels.append(0)
    else:
      labels.append(1)
    # Calculate image feature
    feature = EF(path_to_files + '/'+ file_name, op)
    # -> Insert into list of features
    features.append(feature)

  features = np.array(features)
  labels = np.array(labels)
  return features, labels

In [104]:
op = 0
X_train, y_train = ReadData(dir_datatrain, op)
X_test, y_test = ReadData(dir_datatest, op)

In [105]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1]))

In [106]:
print(X_train.shape)
print(y_train.shape)

(42, 256)
(42,)


In [107]:
print(X_test.shape)
print(y_test.shape)

(20, 256)
(20,)


In [108]:
model = KNeighborsClassifier(n_neighbors=3)
model.fit(X_train, y_train)
acc = model.score(X_test, y_test)
acc

0.45

In [109]:
list = []
for i in range(1,25):
  model = KNeighborsClassifier(n_neighbors=i)
  model.fit(X_train, y_train)
  acc = model.score(X_test, y_test)
  list.append(acc)

print(f"The best K = {list.index(max(list))}    Accuracy = {max(list)}")

The best K = 5    Accuracy = 0.65


# Yêu cầu áp dụng Linear classification cho đặc trưng Histogram

In [110]:
op = 1
X_train, y_train = ReadData(dir_datatrain, op)
X_test, y_test = ReadData(dir_datatest, op)

In [111]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1]))

In [112]:
LoRe = LogisticRegression(random_state=0).fit(X_train, y_train)

In [113]:
score = LoRe.score(X_test, y_test)
score

0.55